In [1]:
#Misc
import scipy.io as sio
import os #to access files from the directories
import math
import time #to check total time took for running the script or function

#Visualization
import matplotlib.pyplot as plt

#Analysis
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import permutation_test_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle
from sklearn.naive_bayes import GaussianNB
from sklearn import tree


In [6]:
dataFolder =  "imp/Data/"
columns = ["means","label"]
#df = pd.DataFrame({'A':1,'B':2}, index = None)
data_list = []
for matFile in os.listdir(dataFolder):
    if matFile.startswith("Faces") and not matFile.endswith("389.mat"):
        data = sio.loadmat(dataFolder + matFile)
        for i in range (len(data["means"])):
        #data_list.append(list(zip(data["means"],data["label"][0]))) # Check this how to do it
            d = data["means"][i],data["label"][0][i]
            data_list.append(d)
df = pd.DataFrame(data_list, columns = columns)
RoiNames = (data["RoiName"][:,0])
colRoi = []
for roi in RoiNames:
    colRoi.append(roi[0])
df[colRoi] = pd.DataFrame(df.means.values.tolist(), index = df.index)
df.drop(['means'], axis=1, inplace=True)

In [20]:
#Extracting different labels entries
df1 = df[df.label == 1]
df2 = df[df.label == 2]
df3 = df[df.label == 3]

In [31]:
#Combining two pairs off all combination
df12 = shuffle(df1.append(df2))
df23 = shuffle(df2.append(df3))
df31 = shuffle(df3.append(df1))


In [72]:

np.where(np.asanyarray(np.isnan(df12)))
df12.iloc[3,94]
#df12.columns[3]
#df12.loc[94,'Frontal_Sup_L']
#print(df12.iloc[3])
#print(df12.columns[94])
#print(df12.loc[4,'Cerebelum_Crus2_R'])
#print(df12.iloc[np.where(np.asanyarray(np.isnan(df12)))])
#print(df12.iloc[564,94])
df12 = df12[df12.label == 1].fillna(df12[df12.label == 1].mean())
np.where(np.asanyarray(np.isnan(df12)))

(array([], dtype=int64), array([], dtype=int64))

In [111]:
df = pd.DataFrame(np.random.randn(5,3))
df.iloc[3:5,0] = np.nan
l = pd.DataFrame({"label":[1,2,1,2,1]})
df = df.join(l)
print(df.mean())
df

0        0.162374
1        0.423409
2       -0.492583
label    1.400000
dtype: float64


,0,1,2,label
0,-0.822863,-0.093227,-0.542361,1
1,0.843451,1.092914,0.059234,2
2,0.466534,-0.513364,-1.439684,1
3,NaN,0.295200,-0.018071,2
4,NaN,1.335522,-0.522031,1


In [119]:
#Mean replacement for nan numbers in particular label with the mean of non missing values in the same label.
tdf = pd.DataFrame(columns=df.columns)
for l in df.label.unique():
    tdf = tdf.append(df[df.label == l].fillna(df[df.label == l].mean()))
    
tdf

,0,1,2,label
0,-0.822863,-0.093227,-0.542361,1
2,0.466534,-0.513364,-1.439684,1
4,-0.178164,1.335522,-0.522031,1
1,0.843451,1.092914,0.059234,2
3,0.843451,0.295200,-0.018071,2


In [114]:

print(df[df.label == 1].mean())
print(df[df.label == 2].mean())
df1 = df[df.label == 1].fillna(df[df.label == 1].mean()) 
df2 = df[df.label == 2].fillna(df[df.label == 2].mean()) 
df = df1.append(df2)
df

0       -0.178164
1        0.242977
2       -0.834692
label    1.000000
dtype: float64
0        0.843451
1        0.694057
2        0.020582
label    2.000000
dtype: float64


,0,1,2,label
0,-0.822863,-0.093227,-0.542361,1
2,0.466534,-0.513364,-1.439684,1
4,-0.178164,1.335522,-0.522031,1
1,0.843451,1.092914,0.059234,2
3,0.843451,0.295200,-0.018071,2
